<a href="https://colab.research.google.com/github/Rys053/Test/blob/main/Fire_detection_with_YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("phylake1337/fire-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'fire-dataset' dataset.
Path to dataset files: /kaggle/input/fire-dataset


In [2]:
import numpy as np
import pandas as pd

import cv2
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import glob
import yaml
import shutil
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm import tqdm

/kaggle/input/fire-dataset/fire_dataset/non_fire_images/non_fire.154.png
/kaggle/input/fire-dataset/fire_dataset/non_fire_images/non_fire.115.png
/kaggle/input/fire-dataset/fire_dataset/non_fire_images/non_fire.59.png
/kaggle/input/fire-dataset/fire_dataset/non_fire_images/non_fire.58.png
/kaggle/input/fire-dataset/fire_dataset/non_fire_images/non_fire.166.png
/kaggle/input/fire-dataset/fire_dataset/non_fire_images/non_fire.142.png
/kaggle/input/fire-dataset/fire_dataset/non_fire_images/non_fire.44.png
/kaggle/input/fire-dataset/fire_dataset/non_fire_images/non_fire.11.png
/kaggle/input/fire-dataset/fire_dataset/non_fire_images/non_fire.19.png
/kaggle/input/fire-dataset/fire_dataset/non_fire_images/non_fire.169.png
/kaggle/input/fire-dataset/fire_dataset/non_fire_images/non_fire.242.png
/kaggle/input/fire-dataset/fire_dataset/non_fire_images/non_fire.29.png
/kaggle/input/fire-dataset/fire_dataset/non_fire_images/non_fire.182.png
/kaggle/input/fire-dataset/fire_dataset/non_fire_images/n

In [5]:
# 디렉토리
dataset_dir = "/kaggle/input/fire-dataset/fire_dataset"
train_image_dir = "/kaggle/working/dataset/train/images"
train_label_dir = "/kaggle/working/dataset/train/labels"
val_image_dir = "/kaggle/working/dataset/val/images"
val_label_dir = "/kaggle/working/dataset/val/labels"

# 디렉토리 만들기
os.makedirs(train_image_dir, exist_ok=True)
os.makedirs(train_label_dir, exist_ok=True)
os.makedirs(val_image_dir, exist_ok=True)
os.makedirs(val_label_dir, exist_ok=True)

train_viz_dir = "/kaggle/working/dataset/train/visualizations"
val_viz_dir = "/kaggle/working/dataset/val/visualizations"
os.makedirs(train_viz_dir, exist_ok=True)
os.makedirs(val_viz_dir, exist_ok=True)

In [6]:
# dataset.yaml 만들기
yaml_content = f"""
path: /kaggle/working/dataset
train: train/images
val: val/images
nc : 1
names: ['fire']
"""
with open("/kaggle/working/dataset/dataset.yaml", "w") as f:
  f.write(yaml_content)

In [7]:
fire_images = [os.path.join(dataset_dir, "fire_images", f) for f in os.listdir(os.path.join(dataset_dir, "fire_images"))]
non_fire_images = [os.path.join(dataset_dir, "non_fire_images", f) for f in os.listdir(os.path.join(dataset_dir, "non_fire_images"))]

all_images = fire_images + non_fire_images
all_labels = [0] * len(fire_images) + [1] * len(non_fire_images)

def is_image_valid(image_path):
  try:
    image = cv2.imread(image_path)
    return image is not None and image.size > 0
  except Exception as e:
    print(f"Error reading image {image_path}: {e}")
    return False

In [8]:
valid_images = []
valid_labels = []
for img, label in zip(all_images, all_labels):
  if is_image_valid(img):
    valid_images.append(img)
    valid_labels.append(label)

print(f"Number of valid images: {len(valid_images)}")

train_images, val_images, train_labels, val_labels = train_test_split(valid_images, valid_labels, test_size=0.2, random_state=42)

Number of valid images: 999


In [9]:
def create_mask(image_path):
  image = cv2.imread(image_path)
  ycbcr = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
  lower_bound = np.array([90, 16, 165])
  upper_bound = np.array([220, 240, 220])
  mask = cv2.inRange(ycbcr, lower_bound, upper_bound)
  return mask

In [10]:
# 색 기반 마스크 생성 및 YOLO segmentation 형식 만들기
def create_mask_and_label(image_path, label):
    # 이미지를 BGR로 읽고 크기를 가져옴
    image = cv2.imread(image_path)
    # height, width는 좌표 정규화에 사용
    height, width = image.shape[:2]

    # BGR -> HSV 색 공간 변환
    # HSV는 색상(Hue), 채도(Saturation), 명도(Value)를 분리하기 때문에 색 기반 검출에 더 직관적임
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # 불꽃 특성을 잡기 위해 주황, 빨강 계열을 선택
    # 채도/ 명도 최소값(100, 100)으로 밝고 채도 높은 픽셀만 검출
    lower_bound = np.array([0, 100, 100])
    upper_bound = np.array([20, 255, 255])

    # 지정한 범윙 안의 픽셀은 흰색, 아닌 경우 검정으로 만드는 이진 마스크 생성
    mask = cv2.inRange(hsv, lower_bound, upper_bound)

    # 마스크 후처리 단계
    # MORPH_CLOSE는 작은 구멍 메우기(노이즈 제거 + 영역 연결)\
    # MORPH_OPEN는 작은 점/잡음 제거
    # 결과적으로 불꽃 영역이 더 깨끗하게 추출됨
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    label_content = []

    for contour in contours:
      # 너무 작은 노이즈는 무시함
      if cv2.contourArea(contour) > 50:
        normalized_polygon = []
        for point in contour.squeeze():
          # 정규화
          x_norm = point[0] / width
          y_norm = point[1] / height
          normalized_polygon.extend([x_norm, y_norm])

        label_line = f"0 {' '.join(map(str, normalized_polygon))}"
        label_content.append(label_line)

    return image, mask, label_content

In [11]:
# 마스크를 원본 이미지 위헤 겹쳐서 시각화(overlay)
def visualize_mask(image, mask):
    # 원본 이미지와 동일한 크기의 0 배열(검은색)
    colored_mask = np.zeros_like(image)
    # 객체가 있는 픽셀에 값 할당, 빨간색
    colored_mask[mask > 0] = [0, 0, 255]

    # 원본과 마스크를 50:50 비율로 섞음
    alpha = 0.5
    # 두 이미지를 가중합으로 합성
    # 원본 이미지 위에 반투명 빨간색 영역이 입혀짐
    visualization = cv2.addWeighted(image, 1 - alpha, colored_mask, alpha, 0)

    # 마스크가 시각적으로 강조된 이미지 반환
    return visualization

In [12]:
# 가공된 이미지, 라벨, 마스크, 시각화 결과를 파일로 저장하는 데이터셋 생성 파이프라인
def save_processed_data(images, labels, image_dir, label_dir, viz_dir):
    for img_path, label in tqdm(zip(images, labels), total=len(images)):
        # 이미지 경로에서 파일명만 추출
        # 예시로 example.jpg -> filename="example.jpg", name_without_ext="example"
        filename = os.path.basename(img_path)
        name_without_ext = os.path.splitext(filename)[0]

        # image: 원본 이미지 (BGR), mask: 불꽃 마스크(이진 흑백 이미지), label_content: YOLO segmentation 형식 라벨 문자열 리스트
        image, mask, label_content = create_mask_and_label(img_path, label)

        # 전처리된 원본 이미지를 저장
        cv2.imwrite(os.path.join(image_dir, filename), image)

        # YOLO 포멧 라벨을 .txt 파일로 저장
        # segmenatation 학습 시 YOLO는 images/train/*.jpg 와 labels/train/*.txt를 매칭시켜 사용
        label_file_path = os.path.join(label_dir, f"{name_without_ext}.txt")
        with open(label_file_path, 'w') as f:
            for line in label_content:
                f.write(line + '\n')

        # 생성된 마스크 이미지를 별도로 저장(.png)
        # 나중에 디버깅, 검증, 데이터 분석 시 활용 가능
        cv2.imwrite(os.path.join(label_dir, f"{name_without_ext}_mask.png"), mask)

        # label_content이 비어 있지 않을 때 시각화 결과 생성
        if len(label_content) > 0:
            # 원본 + 반투명 마스크를 합성
            mask_visualization = visualize_mask(image, mask)
            # .jpg 파일로 저장해서 사람이 눈으로 쉽게 검증할 수 있도록
            cv2.imwrite(os.path.join(viz_dir, f"{name_without_ext}_masked.jpg"), mask_visualization)

In [13]:
# save_processed_data() 함수를 training data와 validation data에 각각 적용해서 YOLO 학습 데이터셋을 최종적으로 준비
print("Saving training data...")
save_processed_data(train_images, train_labels, train_image_dir, train_label_dir, train_viz_dir)

print("Saving validation data...")
save_processed_data(val_images, val_labels, val_image_dir, val_label_dir, val_viz_dir)

print("Data preparation complete!")

Saving training data...


100%|██████████| 799/799 [01:52<00:00,  7.09it/s]


Saving validation data...


100%|██████████| 200/200 [00:28<00:00,  7.03it/s]

Data preparation complete!


In [14]:
print("Starting YOLOv11-Segmentation training...")
!pip install ultralytics
from ultralytics import YOLO

yolo_model = YOLO('yolo11n-seg.pt')

model_config = {
    'data': '/kaggle/working/dataset/dataset.yaml',
    'epochs': 20,
    'batch': 8,
    'imgsz': 640,
    'name': 'fire-detection-yolo-seg',
    'save_period': 10
}

yolo_model.train(**model_config)

print("Training complete!")

Starting YOLOv11-Segmentation training...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.2 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.196 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/dataset/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=